In [2]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Dataset parameters
dataset_path = 'D:/Projects/AIML/Image-detection-AI-MODEL-main/Image-detection-AI-MODEL-main/Dataset'  # Replace with your dataset path
IMG_SIZE = 64
BATCH_SIZE = 32

# Data loading and preprocessing
def load_data(dataset_path, img_size=IMG_SIZE):
    images = []
    labels = []
    
    for label_dir in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label_dir)
        if os.path.isdir(label_path):
            for img_file in os.listdir(label_path):
                img_path = os.path.join(label_path, img_file)
                img = cv2.imread(img_path)
                img = cv2.resize(img, (img_size, img_size))  # Resize image
                images.append(img)
                labels.append(label_dir)
    
    images = np.array(images) / 255.0  # Normalize images
    labels = np.array(pd.factorize(labels)[0])  # Convert labels to integers
    return images, labels

images, labels = load_data(dataset_path)

# Split data
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)


C:\Users\santh\AppData\Local\Temp\ipykernel_10272\3462630793.py:30: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  labels = np.array(pd.factorize(labels)[0])  # Convert labels to integers


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_cnn():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train CNN model
cnn_model = build_cnn()
cnn_model.fit(train_generator, epochs=20, validation_data=(X_test, y_test))
y_pred_cnn = np.argmax(cnn_model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred_cnn))
cnn_model.save('cnn_model.h5')

c:\Users\santh\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


c:\Users\santh\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 298ms/step - accuracy: 0.0429 - loss: 3.1826 - val_accuracy: 0.0496 - val_loss: 3.1333
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step - accuracy: 0.0381 - loss: 3.1330 - val_accuracy: 0.0780 - val_loss: 3.1312
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 186ms/step - accuracy: 0.0500 - loss: 3.1369 - val_accuracy: 0.0780 - val_loss: 3.1327
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 234ms/step - accuracy: 0.0826 - loss: 3.1244 - val_accuracy: 0.0780 - val_loss: 3.1276
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 222ms/step - accuracy: 0.0496 - loss: 3.1173 - val_accuracy: 0.0780 - val_loss: 3.1309
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 231ms/step - accuracy: 0.0594 - loss: 3.1301 - val_accuracy: 0.0780 - val_loss: 3.1286
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.0455 - loss: 3.1243 - val_accuracy: 0.0780 - val_loss: 3.1287
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 146ms/step - accuracy: 0.0570 - loss: 3.1314 - val_accuracy: 0.0709 - val

c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.29      0.20      0.24        10
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        11
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         8
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00         7
          14       0.03      0.75      0.06         4
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         7
          17       0.00    

In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

def build_vgg16():
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train VGG16 model
vgg16_model = build_vgg16()
vgg16_model.fit(train_generator, epochs=20, validation_data=(X_test, y_test))
y_pred_vgg16 = np.argmax(vgg16_model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred_vgg16))
vgg16_model.save('vgg16_model.h5')

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.0423 - loss: 3.4804 - val_accuracy: 0.0709 - val_loss: 3.1059
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 18s 992ms/step - accuracy: 0.0717 - loss: 3.0520 - val_accuracy: 0.0993 - val_loss: 3.0582
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 17s 938ms/step - accuracy: 0.1330 - loss: 2.9945 - val_accuracy: 0.0709 - val_loss: 3.0267
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.1142 - loss: 2.9246 - val_accuracy: 0.1064 - val_loss: 3.0022
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - accuracy: 0.1482 - loss: 2.8503 - val_accuracy: 0.1560 - val_loss: 2.9353
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - accuracy: 0.1726 - loss: 2.7964 - val_accuracy: 0.1277 - val_loss: 2.9834
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.1768 - loss: 2.6819 - val_accuracy: 0.1560 - val_loss: 2.9281
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.2108 - loss: 2.6805 - val_accuracy: 0.1631 - val

c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.29      0.50      0.36         4
           1       0.12      0.20      0.15        10
           2       0.40      0.18      0.25        11
           3       0.18      0.33      0.24         6
           4       0.29      0.18      0.22        11
           5       0.00      0.00      0.00         3
           6       0.29      0.29      0.29         7
           7       0.50      0.17      0.25         6
           8       0.00      0.00      0.00         6
           9       0.11      0.12      0.12         8
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         1
          12       0.43      0.67      0.52         9
          13       0.00      0.00      0.00         7
          14       0.07      0.25      0.11         4
          15       0.00      0.00      0.00         4
          16       0.40      0.29      0.33         7
          17       0.00    

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout

def build_resnet50():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = Sequential([
        base_model,
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train ResNet50 model
resnet50_model = build_resnet50()
resnet50_model.fit(train_generator, epochs=20, validation_data=(X_test, y_test))
y_pred_resnet50 = np.argmax(resnet50_model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred_resnet50))
resnet50_model.save('resnet50_model.h5')

Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.0369 - loss: 3.3183 - val_accuracy: 0.0496 - val_loss: 3.1258
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 810ms/step - accuracy: 0.0509 - loss: 3.1516 - val_accuracy: 0.0355 - val_loss: 3.1163
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 719ms/step - accuracy: 0.0564 - loss: 3.1371 - val_accuracy: 0.0496 - val_loss: 3.1216
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 14s 712ms/step - accuracy: 0.0717 - loss: 3.1445 - val_accuracy: 0.0355 - val_loss: 3.1258
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 791ms/step - accuracy: 0.0597 - loss: 3.1215 - val_accuracy: 0.0355 - val_loss: 3.1115
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 16s 837ms/step - accuracy: 0.0458 - loss: 3.1342 - val_accuracy: 0.0993 - val_loss: 3.1256
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 12s 615ms/step - accuracy: 0.0590 - loss: 3.1279 - val_accuracy: 0.0284 - val_loss: 3.1209
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 15s 713ms/step - accuracy: 0.0788 - loss: 3.1067 - val_accurac

4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001EDD8EB20C0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step  


c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00         6
           4       0.09      1.00      0.16        11
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         8
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00         7
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         7
          17       0.00    

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.layers import TimeDistributed, Conv2D, MaxPooling2D
import numpy as np

# Preparing the data for ConvLSTM
# ConvLSTM expects a 5D tensor: (samples, timesteps, rows, cols, channels)
# Here, we simulate "timesteps" by adding a dummy dimension

def prepare_data_for_convlstm(images, timesteps=1):
    # Adding a time dimension for ConvLSTM (assuming static images treated as one timestep each)
    images = np.expand_dims(images, axis=1)  # Shape: (samples, timesteps, rows, cols, channels)
    return images

X_train_lstm = prepare_data_for_convlstm(X_train)
X_test_lstm = prepare_data_for_convlstm(X_test)

# Model Definition
def build_convlstm(input_shape):
    model = Sequential([
        # ConvLSTM layer
        ConvLSTM2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape, return_sequences=False),
        BatchNormalization(),
        
        # Fully connected layers
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(np.unique(labels)), activation='softmax')  # Output layer with softmax
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the model
input_shape = (X_train_lstm.shape[1], IMG_SIZE, IMG_SIZE, 3)  # (timesteps, rows, cols, channels)
convlstm_model = build_convlstm(input_shape)

# Train the model
history = convlstm_model.fit(
    X_train_lstm, y_train,
    epochs=20,
    batch_size=BATCH_SIZE,
    validation_data=(X_test_lstm, y_test)
)

# Evaluate the model
y_pred_lstm = np.argmax(convlstm_model.predict(X_test_lstm), axis=1)
print("Classification Report:\n", classification_report(y_test, y_pred_lstm))
print("Accuracy Score:", accuracy_score(y_test, y_pred_lstm))

# Save the model
convlstm_model.save('convlstm_model.h5')

c:\Users\santh\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.0554 - loss: 7.7077 - val_accuracy: 0.0709 - val_loss: 3.1542
Epoch 2/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.0398 - loss: 3.1346 - val_accuracy: 0.0780 - val_loss: 3.1340
Epoch 3/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - accuracy: 0.0499 - loss: 3.1353 - val_accuracy: 0.0780 - val_loss: 3.1334
Epoch 4/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.0827 - loss: 3.1335 - val_accuracy: 0.0780 - val_loss: 3.1329
Epoch 5/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.0664 - loss: 3.1337 - val_accuracy: 0.0780 - val_loss: 3.1324
Epoch 6/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.0582 - loss: 3.1333 - val_accuracy: 0.0780 - val_loss: 3.1321
Epoch 7/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 0.0658 - loss: 3.1324 - val_accuracy: 0.0780 - val_loss: 3.1315
Epoch 8/20
18/18 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.0760 - loss: 3.1308 - val_accuracy: 0.0780 - val_loss:

c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        11
           3       0.00      0.00      0.00         6
           4       0.08      1.00      0.14        11
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00         6
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         8
          10       0.00      0.00      0.00         6
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         9
          13       0.00      0.00      0.00         7
          14       0.00      0.00      0.00         4
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         7
   

In [7]:
# Summarizing model performance
model_reports = {
    "CNN": classification_report(y_test, y_pred_cnn, output_dict=True),
    "VGG16": classification_report(y_test, y_pred_vgg16, output_dict=True),
    "ResNet50": classification_report(y_test, y_pred_resnet50, output_dict=True),
    "ConvLSTM": classification_report(y_test, y_pred_lstm, output_dict=True),
}

best_model_name = max(model_reports, key=lambda x: model_reports[x]['accuracy'])
print(f"The best model is {best_model_name}.")

The best model is VGG16.


c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\santh\Lib\site-packages\sklearn\metrics\_classification.py:15